####  PydanticOutputParser 的作用是，将大语言模型（LLM）返回的、非结构化的文本字符串，强制转换成一个你预先定义好的、具有特定结构的、经过验证的 Python 对象。


In [6]:
from langchain_community.chat_models import ChatTongyi

chat = ChatTongyi()

In [9]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, field_validator  # 使用新版本

# 定义数据结构
class Joke(BaseModel):
    setup: str = Field(description="开启一个笑话的问题")
    punchline: str = Field(description="解答笑话的答案")

    # 新版本的验证器语法
    @field_validator("setup")
    @classmethod
    def question_ends_with_question_mark(cls, v):
        if v[-1] != "？":
            raise ValueError("Badly formed question!")
        return v

# 创建解析器
parser = PydanticOutputParser(pydantic_object=Joke)

# 创建提示模板
prompt = PromptTemplate(
    template="根据用户的输入进行解答.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 创建链
chain = prompt | chat | parser
result = chain.invoke({"query": "讲一个笑话"})
print(result)

setup='为什么数学书总是很忧郁？' punchline='因为它有太多的问题。'


In [10]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"description": "开启一个笑话的问题", "title": "Setup", "type": "string"}, "punchline": {"description": "解答笑话的答案", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}\n```'